# Inteligentná analýza údajov 2019/2020 - Projekt
## Druhá časť - predspracovanie
## Ondrej Jedinák, František Tibenský

In [93]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import display

pd.options.display.max_colwidth=130
pd.set_option("display.max_columns", None)

## Konfigurácia

In [94]:
IN_PERSONAL = "data/personal_train.csv"
IN_OTHER = "data/other_train.csv"
OUT = "preprocessed.csv"
DATUM = "now"

## ---------------------

In [95]:
personal_data = pd.read_csv(IN_PERSONAL, index_col=0)
data = pd.read_csv(IN_OTHER, index_col=0)

In [96]:
for ds in [personal_data, data]:
    cls=ds.columns
    cl2=[]
    for cl in cls:
        cl2.append(cl.replace("-","_"))
    ds.rename(columns={i:j for i,j in zip(cls,cl2)}, inplace=True)

Nastavíme konzistentné názvy atribútov pre lepšiu prehľadnosť.

In [97]:
for ds in [data, personal_data]:
    for col in ds:
        if ds[col].dtype=='object':
            ds[col]=ds[col].str.strip()

data=data.replace('?',np.NaN)
data=data.replace('??',np.NaN)

Odstránime zo všetkých textových atribútov prázdne znaky na začiatku a konci a zmeníme hodnoty s otáznikmi na NaN.

In [98]:
display(personal_data.head())
data.head()

,name,address,age,sex,date_of_birth
0,Roscoe Bohannon,"7183 Osborne Ways Apt. 651\r\nEast Andrew, OH 53211",58,Male,1961-03-11
1,Ernest Kline,"391 Ball Road Suite 961\r\nFlowersborough, IN 57550",56,Female,1962/11/20
2,Harold Hendriks,"8702 Vincent Square\r\nNew Jerryfurt, CO 30614",59,Male,11/12/1959
3,Randy Baptiste,"2751 Harris Crossroad\r\nWest Ashley, CA 30311",66,Female,1953-03-18
4,Anthony Colucci,"904 Robert Cliffs Suite 186\r\nWest Kyle, CO 77139",64,Male,1954-12-12


,name,address,race,marital_status,occupation,pregnant,education_num,relationship,skewness_glucose,mean_glucose,capital_gain,kurtosis_glucose,education,fnlwgt,class,std_glucose,income,medical_info,native_country,hours_per_week,capital_loss,workclass
0,Ernest Valles,"698 Miller Gateway Apt. 970\r\nClayside, ND 13457",White,Married-civ-spouse,Adm-clerical,f,10.0,Husband,0.198684,105.859375,0.0,0.222954,Some-college,343789.0,0.0,48.744348,>50K,"{'mean_oxygen':'0.790133779','std_oxygen':'10.13249696','kurtosis_oxygen':'17.68151382','skewness_oxygen':'374.9257754'}",United-States,40.0,0.0,State-gov
1,Michael Kowal,"4784 Walters Extensions\r\nEast Whitney, NH 27458",White,NaN,Sales,F,13.0,Not-in-family,-0.334631,120.453125,0.0,-0.049047,Bachelors,188957.0,0.0,50.982512,<=50K,"{'mean_oxygen':'18.69147157','std_oxygen':'51.21049055','kurtosis_oxygen':'2.541428789','skewness_oxygen':'4.930561896'}",United-States,40.0,0.0,Self-emp-not-inc
2,Mike Rogers,"068 Baker Estates Apt. 300\r\nRoberttown, NV 50205",White,Married-civ-spouse,Sales,f,13.0,Husband,0.030878,117.257812,0.0,0.121774,Bachelors,112283.0,0.0,42.893263,<=50K,"{'mean_oxygen':'1.150501672','std_oxygen':'11.49825164','kurtosis_oxygen':'16.48789423','skewness_oxygen':'320.6702352'}",United-States,55.0,0.0,Private
3,Marvin Cherry,"75326 Odom Drive\r\nNorth Robert, SC 99612",White,Married-civ-spouse,Farming-fishing,f,9.0,Husband,0.636882,99.429688,0.0,0.349696,HS-grad,166416.0,0.0,43.658777,<=50K,"{'mean_oxygen':'0.387123746','std_oxygen':'8.683190316000001','kurtosis_oxygen':'25.389201','skewness_oxygen':'685.06342239999...",United-States,99.0,0.0,Self-emp-not-inc
4,Bernard Williams,"258 Valerie Throughway\r\nSouth Samanthafort, TX 69130",Black,Widowed,Other-service,f,700.0,Not-in-family,0.105785,109.382812,0.0,0.232842,11th,462440.0,0.0,48.991568,<=50K,"{'mean_oxygen':'0.945652174','std_oxygen':'12.0495586','kurtosis_oxygen':'14.8422644','skewness_oxygen':'245.358921'}",United-States,20.0,0.0,Private


## date_of_birth

Tento atribút má nekonzistentné formáty dátumov.

Možne formáty dátumov:

1. yyyy-mm-dd
1. yyyy/mm/dd
1. yy-mm-dd
1. dd/mm/yyyy
1. yyyy-mm-dd 00:00:00
1. yyyy-mm-dd 00 00 00


In [99]:
personal_data["date_of_birth"]=personal_data["date_of_birth"].str.replace(" .*","")
personal_data["date_of_birth"]=personal_data["date_of_birth"].str.replace("/","-")
personal_data['date_of_birth']=personal_data['date_of_birth'].str.replace(r'([0-9]{2})-([0-9]{2})-([0-9]{4})', r'\3-\2-\1')

personal_data['date_of_birth']=personal_data['date_of_birth'].str.replace(r'^(19-[0-9]{2}-[0-9]{2})',r'19\1')
personal_data['date_of_birth']=personal_data['date_of_birth'].str.replace(r'^([2-9][0-9]-[0-9]{2}-[0-9]{2})',r'19\1')
personal_data['date_of_birth']=personal_data['date_of_birth'].str.replace(r'^([0-1][0-9]-[0-9]{2}-[0-9]{2})',r'20\1')

chceme všetky dátumy dostať do 1. formátu
- odstránime časové hodnoty
- zmeníme všetky / na -
- prehodíme rok a deň pri 4. prípade
- doplníme prvé dve cifry roku pre 3. prípad

In [100]:
personal_data["date_of_birth"].str.contains("(^[0-9]{4}-((0[0-9])|(1[0-2]))-([0-2][0-9]|(3)[0-1])$)").unique()

D:\Python\lib\site-packages\ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


array([ True])

Kontrola či sú formáty jednotné.

## age

Zistíme rozdiely vo vekoch a dátumoch narodenia.

In [101]:
((pd.to_datetime(DATUM)-pd.to_datetime(personal_data["date_of_birth"])).astype('<m8[Y]')-personal_data["age"]).value_counts()

0.0    3622
1.0     311
dtype: int64

V dátumoch narodenia a vekoch nie sú žiadne výrazné nezrovnalosti. Jediný problém je že niektoré veky neboli aktualizované pre aktuálny dátum. Tiež neexistuje žiaden dátum pre ktorý by neboli aspoň niektoré veky o jeden rok posunuté. Preto si nanovo vypočítame veky k jednotnému dátumu.

In [102]:
personal_data["age"]=((pd.to_datetime(DATUM)-pd.to_datetime(personal_data["date_of_birth"])).astype('<m8[Y]'))

Keďže nám bude ďalej celý dátum narodenia zbytočný lebo máme vek, ponecháme len mesiac narodenia.

In [103]:
personal_data['date_of_birth']=personal_data['date_of_birth'].str.replace(r'([0-9]{4})-([0-9]{2})-([0-9]{2})', r'\2').astype(str).astype(int)
personal_data.rename(columns={"date_of_birth": "birth_month"},inplace=True)

## workclass

Opravíme nekonzistentné začiatočné písmená.

In [104]:
data["workclass"]=data["workclass"].str.capitalize()
data["workclass"].value_counts(dropna=False)

Private             2737
Self-emp-not-inc     304
Local-gov            266
NaN                  256
State-gov            146
Self-emp-inc         145
Federal-gov          125
Without-pay            3
Never-worked           1
Name: workclass, dtype: int64

## pregnant

In [105]:
data["pregnant"]=data["pregnant"].str.lower()
data["pregnant"]=data["pregnant"].str.replace("(^f$)|(^false$)","False")
data["pregnant"]=data["pregnant"].str.replace("(^t$)|(^true$)","True")
data["pregnant"].value_counts(dropna=False)

False    3888
True       83
NaN        12
Name: pregnant, dtype: int64

Nekonzistentné hodnoty zmeníme tak aby boli len True alebo False (a NaN).

## rozdelenie medical_data

In [106]:
data['medical_info']=data['medical_info'].replace(np.NaN,"{}")
data['medical_info']=data['medical_info'].str.replace("'",'"')
data=data.join(data['medical_info'].apply(json.loads).apply(pd.Series).astype(float))
data = data.drop(columns='medical_info')

Rozparsujeme hodnoty v JSON v medcal_data na samostatné stĺpce. Najprv však nahradíme NaN prázdnymi objektami a zmeníme uvodzovky aby boli validné pre JSON.

## education

In [107]:
data['education']=data['education'].str.replace("_","-")
data['education'].value_counts()

HS-grad         1279
Some-college     910
Bachelors        661
Masters          207
Assoc-voc        151
Assoc-acdm       136
11th             122
10th             114
7th-8th           92
Prof-school       68
9th               63
12th              47
Doctorate         46
5th-6th           45
1st-4th           14
Preschool         10
Name: education, dtype: int64

Zjednotíme nahradenie medzier v hodnotách.

## education_num

In [108]:
def edu_num_fix(val):
    if val<0:
        val*=-1
    if val>100:
        val/=100
    return val
data['education_num']=data['education_num'].apply(edu_num_fix)
data.education_num.value_counts()

9.0     1151
10.0     827
13.0     589
14.0     185
11.0     141
12.0     125
7.0      109
6.0       99
4.0       82
15.0      59
5.0       59
8.0       45
16.0      41
3.0       39
2.0       13
1.0        6
Name: education_num, dtype: int64

Zjednotíme hodnoty tak aby nám zostali len hodnoty od 1 do 16.

In [109]:
for i in range(1,17):
    print(str(data['education'][data.education_num==i].value_counts()))

Preschool    6
Name: education, dtype: int64
1st-4th    13
Name: education, dtype: int64
5th-6th    39
Name: education, dtype: int64
7th-8th    81
Name: education, dtype: int64
9th    59
Name: education, dtype: int64
10th    99
Name: education, dtype: int64
11th    108
Name: education, dtype: int64
12th    45
Name: education, dtype: int64
HS-grad    1147
Name: education, dtype: int64
Some-college    823
Name: education, dtype: int64
Assoc-voc    141
Name: education, dtype: int64
Assoc-acdm    125
Name: education, dtype: int64
Bachelors    587
Name: education, dtype: int64
Masters    184
Name: education, dtype: int64
Prof-school    59
Name: education, dtype: int64
Doctorate    41
Name: education, dtype: int64


In [110]:
edu={}
for i in range(1,17):
    edu[str(data['education'][data.education_num==i].value_counts()).split(" ")[0]]=i
edu

{'Preschool': 1,
 '1st-4th': 2,
 '5th-6th': 3,
 '7th-8th': 4,
 '9th': 5,
 '10th': 6,
 '11th': 7,
 '12th': 8,
 'HS-grad': 9,
 'Some-college': 10,
 'Assoc-voc': 11,
 'Assoc-acdm': 12,
 'Bachelors': 13,
 'Masters': 14,
 'Prof-school': 15,
 'Doctorate': 16}

Zistili sme že education_num je len číselná verzia eduaction. Doplníme podľa toho chýbajúce hodnoty education_num a stĺpec education môžeme zmazať.

In [111]:
def edu_to_num(val):
    if val in edu:
        return edu[val]
    else:
        return np.NaN
    
data['education']=data['education'].apply(edu_to_num)
data['education'].fillna(data['education_num'], inplace=True)

data['education'].value_counts(dropna=False)
data = data.drop(columns=['education_num'])

## Odstránenie duplikátov

In [112]:
def aggr(val):
    o=set()
    for v in val:
        if not pd.isna(v):
            o.add(v)
    if len(o)==0:
        return np.NaN
    elif len(o)>1:
        print("Pozor: viac moznych hodnot!")
        print(o)
    return o.pop()
    
data=data.groupby(['name', 'address']).aggregate(aggr)

Záznamy s rovnakými menami a adresami zjednotíme a doplníme ich hodnoty pomocou agregačnej funkcie.

## Spojenie dvoch tabuliek

In [113]:
data=data.merge(personal_data, on=('name', 'address'))

In [114]:
data.address=data.address.str.replace(r".*\r\n[A-Za-z ]+,? ([A-Z]{2}).*",r"\1")

In [115]:
data.rename(columns={"address": "state"},inplace=True)

## spojenie capital_loss a capital_gain

In [116]:
data.isnull().sum()

name                  0
state                 0
race                  1
marital_status      393
occupation          238
pregnant              0
relationship          0
skewness_glucose      1
mean_glucose          0
capital_gain          0
kurtosis_glucose      1
education             0
fnlwgt                0
class                 0
std_glucose           0
income                1
native_country       81
hours_per_week        0
capital_loss          0
workclass           238
mean_oxygen           1
std_oxygen            1
kurtosis_oxygen       1
skewness_oxygen       1
age                   0
sex                   0
birth_month           0
dtype: int64

In [117]:
data = data.assign(capital=np.NaN)
data.capital = data.capital_gain - data.capital_loss
data = data.drop(columns=['capital_loss','capital_gain'])

In [118]:
data.head(100)

,name,state,race,marital_status,occupation,pregnant,relationship,skewness_glucose,mean_glucose,kurtosis_glucose,education,fnlwgt,class,std_glucose,income,native_country,hours_per_week,workclass,mean_oxygen,std_oxygen,kurtosis_oxygen,skewness_oxygen,age,sex,birth_month,capital
0,Aaron Davis,MD,White,Never-married,Prof-specialty,False,Not-in-family,25.996764,38.921875,4.751118,14.0,72880.0,1.0,31.821428,<=50K,United-States,40.0,Private,46.650502,66.122251,1.463561,1.411576,57.0,Female,5,0.0
1,Aaron Dini,KS,White,NaN,Prof-specialty,False,Husband,11.255049,61.460938,2.539502,13.0,43235.0,1.0,33.785376,>50K,United-States,50.0,Private,9.411371,39.407883,4.610877,21.099716,53.0,Male,5,0.0
2,Aaron Gann,UT,White,Never-married,Prof-specialty,True,Own-child,0.818221,150.062500,-0.327482,13.0,126613.0,0.0,42.052190,<=50K,United-States,8.0,Private,5.908863,25.735714,4.914792,26.461769,51.0,Female,5,0.0
3,Aaron Gregg,ME,White,Married-civ-spouse,Sales,False,Husband,0.573133,116.796875,0.275827,10.0,42251.0,0.0,46.736689,>50K,United-States,45.0,Private,5.130435,25.819246,5.966746,37.867530,54.0,Male,4,0.0
4,Aaron Lindley,AR,White,Married-civ-spouse,Other-service,False,Husband,37.449172,16.539062,5.971550,9.0,135102.0,1.0,30.500743,<=50K,United-States,45.0,Local-gov,67.451505,67.144170,0.905511,0.064850,54.0,Male,12,-2002.0
5,Aaron Manning,DC,White,Never-married,Adm-clerical,False,Not-in-family,0.203493,126.859375,0.001818,9.0,51047.0,0.0,45.451508,<=50K,United-States,25.0,Private,3.389632,20.382267,7.829814,70.000805,56.0,Female,1,0.0
6,Aaron Moody,MS,White,Married-spouse-absent,Sales,False,Not-in-family,30.428177,31.601562,5.036450,10.0,235556.0,1.0,29.498236,<=50K,Mexico,45.0,Private,33.494983,62.868452,2.061247,3.273828,67.0,Male,3,0.0
7,Aaron Noman,AA,White,Married-civ-spouse,Prof-specialty,False,Husband,0.332530,105.242188,0.112815,15.0,99835.0,0.0,44.526457,>50K,United-States,60.0,Private,3.340301,19.251949,7.984062,75.921964,22.0,Male,10,0.0
8,Aaron Robinson,MD,White,Married-civ-spouse,Prof-specialty,False,Husband,2.081514,85.445312,1.314571,15.0,65324.0,1.0,49.863534,>50K,United-States,40.0,Private,131.379599,71.988884,-0.473466,-0.579712,60.0,Male,5,0.0
9,Aaron Rogers,OH,White,Never-married,Craft-repair,False,Own-child,4.199041,80.468750,1.296000,10.0,138692.0,1.0,36.388300,<=50K,United-States,50.0,Private,46.872910,69.420130,1.347551,0.763489,56.0,Male,8,0.0


TODO:
- doplnit NaN hodnoty podla metod popisanych v zadani (najma tie ciselne).
- pozriet ciselne outliery, nahradzat podla metod v zadani
- nahradit boolean hodnoty 0 a 1.
- doplnit marital-status cez relationship
- zrusit tehotnych muzov
- indexy zoradit naspat ako boli povodne (teraz je to podla abecedy)
- prieskumna analyza pre oxygeny, education, age, birth_month

Otazkzy na cviko: 
- ako doplnit stat?

In [90]:
#new_educ = data["std_glucose"]
"NaN: "+str(data["fnlwgt"].isnull().sum())

'NaN: 14'